# ETL Project - TRANSFORM
## Terrence Cummings
Data: Race mix of Minneapolis neighborhoods scraped from mncompass.org

In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import time
import sys
import os
import datetime
from selenium.webdriver.chrome.options import Options
from sqlalchemy import create_engine
chrome_options = Options()
chrome_options.add_argument("--headless")

In [7]:
#Read in CSV's from Extract
cmnty_links_df=pd.read_csv('cmnty_links.csv')
nbhd_links_df=pd.read_csv('nbhd_links.csv')
cmnty_race_df=pd.read_csv('cmnty_race.csv')
nbhd_race_df=pd.read_csv('nbhd_race.csv')
cmnty_keys_df=pd.read_csv('cmnty_keys.csv')
nbhd_keys_df=pd.read_csv('nbhd_keys.csv')

In [8]:
#Add community key to neighborhood key table to use as a foreign key
nbhd_keys_df = nbhd_keys_df.merge(cmnty_keys_df, how='left', left_on='COMMUNITY', right_on='COMMUNITY')


## Create initial table of MSP neighborhoods (rows) and race mix (columns)

In [9]:
#Some figures from the scraped table show the word 'suppressed'. Replace with NaN for subsequent handling.
nbhd_race_df = nbhd_race_df.replace('suppressed', np.nan)


In [10]:
#Convert population count text to numbers
nbhd_race_df['white_cnt'] = nbhd_race_df['white_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['black_cnt'] = nbhd_race_df['black_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['native_cnt'] = nbhd_race_df['native_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['asian_cnt'] = nbhd_race_df['asian_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['other_cnt'] = nbhd_race_df['other_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['two_or_more_cnt'] = nbhd_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['hispanic_cnt'] = nbhd_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['of_color_cnt'] = nbhd_race_df['of_color_cnt'].str.replace(',', '').astype(float)

In [11]:
#Convert population percentage text to numbers
nbhd_race_df['white_pct'] = nbhd_race_df['white_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['black_pct'] = nbhd_race_df['black_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['native_pct'] = nbhd_race_df['native_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['asian_pct'] = nbhd_race_df['asian_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['other_pct'] = nbhd_race_df['other_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['two_or_more_pct'] = nbhd_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['hispanic_pct'] = nbhd_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['of_color_pct'] = nbhd_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [12]:
#Add total population estimate for each neighborood based on the white count and percentage.
nbhd_race_df['total_cnt']=round(nbhd_race_df['white_cnt']/nbhd_race_df['white_pct'],0)

In [13]:
#Add the neighorhood key and the official neighborhood names to the table. Drop some unnecessary columns.
#The 'outer' merge also adds rows for all the missing MSP neighborhoods not present in the scraped data.
#Inclusion of all neighborhoods and official names will facilitate analysis with the Open Data MSP police record data.
temp_nbhd_race_df = nbhd_race_df.merge(nbhd_keys_df, how='outer', left_on='neighborhood', right_on='neighborhood_url')
final_nbhd_race_df=temp_nbhd_race_df.drop(['neighborhood', 'neighborhood_url'], axis = 1) 
final_nbhd_race_df.set_index('NBHD_KEY', inplace=True)


In [14]:
#Reorder columns for presentability. At this stage have a table with all neighborhoods and races, but with a lot of NaN's that needed to be reconciled.
final_nbhd_race_df = final_nbhd_race_df[['NEIGHBORHOOD','COMMUNITY','total_cnt','white_cnt','white_pct','black_cnt','black_pct','native_cnt','native_pct','asian_cnt','asian_pct','other_cnt','other_pct','two_or_more_cnt','two_or_more_pct','hispanic_cnt','hispanic_pct','of_color_cnt','of_color_pct','URL']]


## Create table of MSP communities (rows) and race mix (columns). This will be used to backfill missing neighborhood-level data.

In [15]:
#Set the few 'suppressed' data to 0 as immaterial. 
cmnty_race_df['community'] = cmnty_race_df['community'].str.upper()
cmnty_race_df = cmnty_race_df.replace('suppressed', '0')
cmnty_race_df = cmnty_race_df.replace(np.nan, '0', regex=True)


In [16]:
#Convert race population count text to numbers.
cmnty_race_df['white_cnt'] = cmnty_race_df['white_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['black_cnt'] = cmnty_race_df['black_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['native_cnt'] = cmnty_race_df['native_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['asian_cnt'] = cmnty_race_df['asian_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['other_cnt'] = cmnty_race_df['other_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['two_or_more_cnt'] = cmnty_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['hispanic_cnt'] = cmnty_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['of_color_cnt'] = cmnty_race_df['of_color_cnt'].str.replace(',', '').astype(float)


In [17]:
#Convert race population percentages to numbers
cmnty_race_df['white_pct'] = cmnty_race_df['white_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['black_pct'] = cmnty_race_df['black_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['native_pct'] = cmnty_race_df['native_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['asian_pct'] = cmnty_race_df['asian_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['other_pct'] = cmnty_race_df['other_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['two_or_more_pct'] = cmnty_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['hispanic_pct'] = cmnty_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['of_color_pct'] = cmnty_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [18]:
#Create a dicationary to map the scraped community names to the official community names
cmnty_dict = {'CALHOUN-ISLES': 'CALHOUN-ISLE', 'CAMDEN':'CAMDEN', 'CENTRAL': 'CENTRAL', 'LONGFELLOW': 'LONGFELLOW', 'NEAR-NORTH': 'NEAR NORTH', 'NOKOMIS':'NOKOMIS', 'NORTHEAST':'NORTHEAST', 'PHILLIPS': 'PHILLIPS', 'POWDERHORN': 'POWDERHORN', 'SOUTHWEST':'SOUTHWEST', 'UNIVERSITY':'UNIVERSITY'}

In [19]:
#Replaced scraped community names with official community names
cmnty_race_df = cmnty_race_df.replace({"community": cmnty_dict})

In [20]:
#Add community keys to the community race table
final_cmnty_race_df = cmnty_race_df.merge(cmnty_keys_df, how='left', left_on='community', right_on='COMMUNITY')
del final_cmnty_race_df['COMMUNITY']
final_cmnty_race_df.set_index('CMNTY_KEY', inplace=True)
final_cmnty_race_df['total_cnt']=final_cmnty_race_df['white_cnt']+final_cmnty_race_df['of_color_cnt']


In [21]:
#Recalculate the race mix percentages based on the population counts
final_cmnty_race_df['white_pct']=final_cmnty_race_df['white_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['black_pct']=final_cmnty_race_df['black_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['native_pct']=final_cmnty_race_df['native_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['asian_pct']=final_cmnty_race_df['asian_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['other_pct']=final_cmnty_race_df['other_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['two_or_more_pct']=final_cmnty_race_df['two_or_more_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['hispanic_pct']=final_cmnty_race_df['hispanic_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df['of_color_pct']=final_cmnty_race_df['of_color_cnt']/final_cmnty_race_df['total_cnt']
final_cmnty_race_df = final_cmnty_race_df[['community','total_cnt','white_cnt','white_pct','black_cnt','black_pct','native_cnt','native_pct','asian_cnt','asian_pct','other_cnt','other_pct','two_or_more_cnt','two_or_more_pct','hispanic_cnt','hispanic_pct','of_color_cnt', 'of_color_pct','URL']]



In [22]:
#Use group the neighborhood table by community. This can then be compared to the community table to understand how much data is missing from the neighborhood table and how to handle NaN's in the neighborhood table.
nbhd_cmnty_grp_df = pd.DataFrame(final_nbhd_race_df.groupby('COMMUNITY').sum())

In [23]:
#Calculate the total population for each community in the grouped table.
nbhd_cmnty_grp_df['total_cnt'] = nbhd_cmnty_grp_df['white_cnt']+nbhd_cmnty_grp_df['of_color_cnt']

In [24]:
#Calculate the community race mix percentages from the grouped table for comparison to the same percentages in the commumity scrape table
nbhd_cmnty_grp_df['white_pct']=nbhd_cmnty_grp_df['white_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['black_pct']=nbhd_cmnty_grp_df['black_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['native_pct']=nbhd_cmnty_grp_df['native_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['asian_pct']=nbhd_cmnty_grp_df['asian_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['other_pct']=nbhd_cmnty_grp_df['other_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['two_or_more_pct']=nbhd_cmnty_grp_df['two_or_more_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['hispanic_pct']=nbhd_cmnty_grp_df['hispanic_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df['of_color_pct']=nbhd_cmnty_grp_df['of_color_cnt']/nbhd_cmnty_grp_df['total_cnt']
nbhd_cmnty_grp_df = nbhd_cmnty_grp_df[['total_cnt','white_cnt','white_pct','black_cnt','black_pct','native_cnt','native_pct','asian_cnt','asian_pct','other_cnt','other_pct','two_or_more_cnt','two_or_more_pct','hispanic_cnt','hispanic_pct','of_color_cnt', 'of_color_pct']]

In [25]:
#Join the scraped community table with the grouped community table to compare differences in race mix estimates
temp_cmnty_join_df = final_cmnty_race_df.merge(nbhd_cmnty_grp_df, how='left', left_on='community', right_index=True)

In [26]:
#Solve for the race mix percentages to be assigned to the NaN's in the neighborhood table in order to reproduce the community level race mix.
temp_cmnty_join_df['white_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['white_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['white_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['black_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['black_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['black_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['native_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['native_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['native_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['asian_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['asian_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['asian_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['other_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['other_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['other_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['two_or_more_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['two_or_more_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['two_or_more_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['hispanic_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['hispanic_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['hispanic_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['of_color_bal_pct'] = (temp_cmnty_join_df['total_cnt_x']*temp_cmnty_join_df['of_color_pct_x']-temp_cmnty_join_df['total_cnt_y']*temp_cmnty_join_df['of_color_pct_y'])/(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])

temp_cmnty_join_df['marg_err']=(temp_cmnty_join_df['total_cnt_x']-temp_cmnty_join_df['total_cnt_y'])/temp_cmnty_join_df['total_cnt_x']

In [27]:
#Delete the colujns no longer needed.
temp_cmnty_join_df=temp_cmnty_join_df.drop(['total_cnt_x','white_cnt_x', 'black_cnt_x', 'native_cnt_x','asian_cnt_x','other_cnt_x','two_or_more_cnt_x','hispanic_cnt_x','of_color_cnt_x','total_cnt_y','white_cnt_y','black_cnt_y','native_cnt_y','asian_cnt_y','other_cnt_y','two_or_more_cnt_y','hispanic_cnt_y','of_color_cnt_y','URL'] , axis = 1) 


In [28]:
#Rename the columns to more explanatory names
temp_cmnty_join_df.rename(columns={'white_pct_x': 'cmnty_white_pct', 'black_pct_x': 'cmnty_black_pct', 'native_pct_x': 'cmnty_native_pct', 'asian_pct_x': 'cmnty_asian_pct', 'other_pct_x': 'cmnty_other_pct', 'two_or_more_pct_x': 'cmnty_two_or_more_pct', 'hispanic_pct_x': 'cmnty_hispanic_pct', 'of_color_pct_x': 'cmnty_of_color_pct', 'white_pct_y': 'nbhd_white_pct', 'black_pct_y': 'nbhd_black_pct', 'native_pct_y': 'nbhd_native_pct', 'asian_pct_y': 'nbhd_asian_pct', 'other_pct_y': 'nbhd_other_pct', 'two_or_more_pct_y': 'nbhd_two_or_more_pct', 'hispanic_pct_y': 'nbhd_hispanic_pct', 'of_color_pct_y': 'nbhd_of_color_pct'}, inplace=True)

In [29]:
#Assign the df a better name.
race_mix_nan_df = temp_cmnty_join_df


In [30]:
#Manual cleanup of some nonsensical results due divide by 'near zero' issues
race_mix_nan_df.iloc[3, race_mix_nan_df.columns.get_loc('black_bal_pct')] = 0
race_mix_nan_df.iloc[3, race_mix_nan_df.columns.get_loc('native_bal_pct')] = 0
race_mix_nan_df.iloc[3, race_mix_nan_df.columns.get_loc('asian_bal_pct')] = 0
race_mix_nan_df.iloc[3, race_mix_nan_df.columns.get_loc('other_bal_pct')] = 0
race_mix_nan_df.iloc[3, race_mix_nan_df.columns.get_loc('two_or_more_bal_pct')] = 0
race_mix_nan_df.iloc[3, race_mix_nan_df.columns.get_loc('hispanic_bal_pct')] = 0
race_mix_nan_df.iloc[3, race_mix_nan_df.columns.get_loc('of_color_bal_pct')] = 0

race_mix_nan_df.iloc[9, race_mix_nan_df.columns.get_loc('white_bal_pct')] = 0
race_mix_nan_df.iloc[9, race_mix_nan_df.columns.get_loc('black_bal_pct')] = 0
race_mix_nan_df.iloc[9, race_mix_nan_df.columns.get_loc('native_bal_pct')] = 0
race_mix_nan_df.iloc[9, race_mix_nan_df.columns.get_loc('asian_bal_pct')] = 0
race_mix_nan_df.iloc[9, race_mix_nan_df.columns.get_loc('other_bal_pct')] = 0
race_mix_nan_df.iloc[9, race_mix_nan_df.columns.get_loc('two_or_more_bal_pct')] = 0


In [31]:
#Create a table of just the columns needed that will replace NaN values in the neighborhood data
race_mix_nan_sub_pct = race_mix_nan_df[['community','white_bal_pct', 'black_bal_pct', 'native_bal_pct', 'asian_bal_pct', 'other_bal_pct', 'two_or_more_bal_pct', 'hispanic_bal_pct', 'of_color_bal_pct']]


In [32]:
#Get rid of unnecessary columns
nbhd_race_mix_df=final_nbhd_race_df.drop(['total_cnt','white_cnt', 'black_cnt', 'native_cnt','asian_cnt','other_cnt','two_or_more_cnt','hispanic_cnt','of_color_cnt','URL'] , axis = 1) 

In [33]:
#Merge the table with the 'balancing percentages' with the original neighborhood table with NaN's
temp_sub_pct_df = nbhd_race_mix_df.merge(race_mix_nan_sub_pct, how='left', left_on='COMMUNITY', right_on='community')

In [34]:
#Where the neighborhood table had NaN's assign that cell a value based on the balancing percentages calculated previously
temp_sub_pct_df['white_pct']=temp_sub_pct_df['white_pct'].fillna(temp_sub_pct_df['white_bal_pct'])
temp_sub_pct_df['black_pct']=temp_sub_pct_df['black_pct'].fillna(temp_sub_pct_df['black_bal_pct'])
temp_sub_pct_df['native_pct']=temp_sub_pct_df['native_pct'].fillna(temp_sub_pct_df['native_bal_pct'])
temp_sub_pct_df['asian_pct']=temp_sub_pct_df['asian_pct'].fillna(temp_sub_pct_df['asian_bal_pct'])
temp_sub_pct_df['other_pct']=temp_sub_pct_df['other_pct'].fillna(temp_sub_pct_df['other_bal_pct'])
temp_sub_pct_df['two_or_more_pct']=temp_sub_pct_df['two_or_more_pct'].fillna(temp_sub_pct_df['two_or_more_bal_pct'])
temp_sub_pct_df['hispanic_pct']=temp_sub_pct_df['hispanic_pct'].fillna(temp_sub_pct_df['hispanic_bal_pct'])
temp_sub_pct_df['of_color_pct']=temp_sub_pct_df['of_color_pct'].fillna(temp_sub_pct_df['of_color_bal_pct'])



In [35]:
#Add the neighborhood key (Primary key)
temp_sub_pct_df = pd.merge(temp_sub_pct_df,nbhd_keys_df[['NEIGHBORHOOD','NBHD_KEY']],on='NEIGHBORHOOD', how='left')

In [36]:
#Add the community key (foreign key)
temp_sub_pct_df = pd.merge(temp_sub_pct_df,cmnty_keys_df[['COMMUNITY','CMNTY_KEY']],on='COMMUNITY', how='left')

## CREATE FINAL TABLE FOR LOADING INTO POSTGRESQL

1. msp_nbhd_race_mix_sql: The percentage of the population of each race in each of the 87 Minneapolis neighborhoods. Indicated by a unique Neighborhood Key.

2. msp_cmnty_keys_sql: A unique Community Key and the community name for each of the 11 Minneapolis communities

3. msp_nbhd_keys_sql: A unique Neighborhood Key, the neighborhood name, and the Community Key for the community that contains the neighborhood for each of the 87 Minneapolis neighborhoods.


In [37]:
#Final neighborhood race mix table.
msp_nbhd_race_mix_sql = temp_sub_pct_df[['NBHD_KEY', 'white_pct','black_pct','native_pct','asian_pct','other_pct','two_or_more_pct','hispanic_pct','of_color_pct']]

In [38]:
#Set table index to the Neighborhood Key
msp_nbhd_race_mix_sql.set_index('NBHD_KEY', inplace=True)

In [39]:
msp_nbhd_race_mix_sql

,white_pct,black_pct,native_pct,asian_pct,other_pct,two_or_more_pct,hispanic_pct,of_color_pct
NBHD_KEY,,,,,,,,
N71,0.845000,0.000000,0.000000,0.000000,0.000000,0.036000,0.896364,0.155000
N46,0.788000,0.075000,0.022493,0.061600,0.092546,0.059000,0.088000,0.212000
N63,0.734000,0.116000,0.244486,0.021000,0.377446,0.078000,0.147000,0.266000
N02,0.916000,0.025483,0.000000,0.036218,0.045110,0.064086,0.055736,0.156474
N80,0.320000,0.511000,0.124708,0.115000,0.119707,0.039000,0.027000,0.680000
...,...,...,...,...,...,...,...,...
N55,0.734049,0.149074,0.022493,0.061600,0.092546,0.044031,0.115922,0.265951
N56,0.734049,0.149074,0.022493,0.061600,0.092546,0.044031,0.115922,0.265951
N33,0.344636,0.397919,0.076247,0.092573,0.092573,0.080732,0.123789,0.655364


In [40]:
#The final Community Key table. 
cmnty_keys_df.set_index('CMNTY_KEY', inplace=True)
msp_cmnty_keys_sql = cmnty_keys_df
msp_cmnty_keys_sql

,COMMUNITY
CMNTY_KEY,
C00,CALHOUN-ISLE
C01,CAMDEN
C02,CENTRAL
C03,LONGFELLOW
C04,NEAR NORTH
C05,NOKOMIS
C06,NORTHEAST
C07,PHILLIPS
C08,POWDERHORN


In [41]:
#The final Neighborhood Key table
msp_nbhd_keys_sql=nbhd_keys_df.drop(['COMMUNITY', 'neighborhood_url'], axis = 1) 
msp_nbhd_keys_sql.set_index('NBHD_KEY', inplace=True)

In [42]:
msp_nbhd_keys_sql

,NEIGHBORHOOD,CMNTY_KEY
NBHD_KEY,,
N71,ARMATAGE,C09
N46,AUDUBON PARK,C06
N63,BANCROFT,C08
N47,BELTRAMI,C06
N48,BOTTINEAU,C06
...,...,...
N08,WEST CALHOUN,C00
N70,WHITTIER,C08
N34,WILLARD-HAY,C04


In [43]:
#Write the final 3 table to csv's
msp_nbhd_race_mix_sql.to_csv('msp_nbhd_race_mix_sql.csv')
msp_cmnty_keys_sql.to_csv('msp_cmnty_keys_sql.csv')
msp_nbhd_keys_sql.to_csv('msp_nbhd_keys_sql.csv')


In [51]:
msp_nbhd_keys_sql

,NEIGHBORHOOD,CMNTY_KEY
NBHD_KEY,,
N71,ARMATAGE,C09
N46,AUDUBON PARK,C06
N63,BANCROFT,C08
N47,BELTRAMI,C06
N48,BOTTINEAU,C06
...,...,...
N08,WEST CALHOUN,C00
N70,WHITTIER,C08
N34,WILLARD-HAY,C04


In [44]:
host = "localhost"
user = "postgres"
port = "5432"
passwd = "hawkeyes"
db = "REP_13_ETL_Project"

engine = create_engine(f'postgresql://{user}:{passwd}@{host}:{port}/{db}')
connection = engine.connect()

In [54]:

msp_nbhd_race_mix_sql.to_sql('msp_nbhd_race_mix', connection, if_exists='replace')

msp_cmnty_keys_sql.to_sql('msp_cmnty_keys', connection, if_exists='replace')
msp_nbhd_keys_sql.to_sql('msp_nbhd_keys', connection, if_exists='replace')


InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table msp_cmnty_keys because other objects depend on it
DETAIL:  constraint fk_msp_nbhd_keys_CMNTY_KEY on table msp_nbhd_keys depends on table msp_cmnty_keys
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE msp_cmnty_keys]
(Background on this error at: http://sqlalche.me/e/2j85)